In [20]:
import pandas as pd
import pickle

In [2]:
#Get MTA Lat-Lon data and create dataframe
mta_loc = pd.read_csv("https://data.cityofnewyork.us/api/views/he7q-3hwy/rows.csv?accessType=DOWNLOAD")
mta_loc.head()

,OBJECTID,URL,NAME,the_geom,LINE
0,1734,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NW corner,POINT (-73.86835600032798 40.84916900104506),2-5
1,1735,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NE corner,POINT (-73.86821300022677 40.84912800131844),2-5
2,1736,http://web.mta.info/nyct/service/,Morris Park Ave & 180th St at NW corner,POINT (-73.87349900050798 40.84122300105249),2-5
3,1737,http://web.mta.info/nyct/service/,Morris Park Ave & 180th St at NW corner,POINT (-73.8728919997833 40.84145300067447),2-5
4,1738,http://web.mta.info/nyct/service/,Boston Rd & 178th St at SW corner,POINT (-73.87962300013866 40.84081500075867),2-5


In [3]:
#Parse out the Lat and Lon Fields
mta_loc["LON"] = mta_loc["the_geom"].str[7:24]
mta_loc["LAT"] = mta_loc["the_geom"].str[25:-1]
mta_loc.head()

,OBJECTID,URL,NAME,the_geom,LINE,LON,LAT
0,1734,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NW corner,POINT (-73.86835600032798 40.84916900104506),2-5,-73.8683560003279,40.84916900104506
1,1735,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NE corner,POINT (-73.86821300022677 40.84912800131844),2-5,-73.8682130002267,40.84912800131844
2,1736,http://web.mta.info/nyct/service/,Morris Park Ave & 180th St at NW corner,POINT (-73.87349900050798 40.84122300105249),2-5,-73.8734990005079,40.84122300105249
3,1737,http://web.mta.info/nyct/service/,Morris Park Ave & 180th St at NW corner,POINT (-73.8728919997833 40.84145300067447),2-5,-73.8728919997833,40.84145300067447
4,1738,http://web.mta.info/nyct/service/,Boston Rd & 178th St at SW corner,POINT (-73.87962300013866 40.84081500075867),2-5,-73.8796230001386,40.84081500075867


In [4]:
#Create new upper case station name column for merging
mta_loc["STATION"] = mta_loc["NAME"].str.upper()
mta_loc.head()

,OBJECTID,URL,NAME,the_geom,LINE,LON,LAT,STATION
0,1734,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NW corner,POINT (-73.86835600032798 40.84916900104506),2-5,-73.8683560003279,40.84916900104506,BIRCHALL AVE & SAGAMORE ST AT NW CORNER
1,1735,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NE corner,POINT (-73.86821300022677 40.84912800131844),2-5,-73.8682130002267,40.84912800131844,BIRCHALL AVE & SAGAMORE ST AT NE CORNER
2,1736,http://web.mta.info/nyct/service/,Morris Park Ave & 180th St at NW corner,POINT (-73.87349900050798 40.84122300105249),2-5,-73.8734990005079,40.84122300105249,MORRIS PARK AVE & 180TH ST AT NW CORNER
3,1737,http://web.mta.info/nyct/service/,Morris Park Ave & 180th St at NW corner,POINT (-73.8728919997833 40.84145300067447),2-5,-73.8728919997833,40.84145300067447,MORRIS PARK AVE & 180TH ST AT NW CORNER
4,1738,http://web.mta.info/nyct/service/,Boston Rd & 178th St at SW corner,POINT (-73.87962300013866 40.84081500075867),2-5,-73.8796230001386,40.84081500075867,BOSTON RD & 178TH ST AT SW CORNER


In [5]:
#Delete unnecessary columns
mta_loc_clean = mta_loc.drop(["OBJECTID", "URL", "NAME", "the_geom", "LINE"], axis=1, errors="ignore")
mta_loc_clean.head()

,LON,LAT,STATION
0,-73.8683560003279,40.84916900104506,BIRCHALL AVE & SAGAMORE ST AT NW CORNER
1,-73.8682130002267,40.84912800131844,BIRCHALL AVE & SAGAMORE ST AT NE CORNER
2,-73.8734990005079,40.84122300105249,MORRIS PARK AVE & 180TH ST AT NW CORNER
3,-73.8728919997833,40.84145300067447,MORRIS PARK AVE & 180TH ST AT NW CORNER
4,-73.8796230001386,40.84081500075867,BOSTON RD & 178TH ST AT SW CORNER


In [6]:
#Dedupe on Station name
#Assume that lat/lon difference for duplicate stations (entrances) is not material when assigning zip code
mta_loc_clean.sort_values(["STATION"], inplace=True, \
                          ascending=False)
mta_loc_clean.drop_duplicates(["STATION"], inplace=True)
mta_loc_clean.head()

,LON,LAT,STATION
992,-73.8709829998929,40.733354001050536,WOODHAVEN BLVD & QUEENS BLVD AT SW CORNER
991,-73.8703340001457,40.73304100076268,WOODHAVEN BLVD & QUEENS BLVD AT SE CORNER
212,-73.8525559995751,40.69345100050823,WOODHAVEN BLVD & JAMAICA AVE AT SW CORNER
213,-73.8526080001453,40.69356300102105,WOODHAVEN BLVD & JAMAICA AVE AT NW CORNER
1336,-74.0096579997829,40.706385000547535,WILLIAM ST & WALL ST AT SW CORNER


In [12]:
#Export MTA station lat/lon data for reverse geocode
mta_loc_clean.to_csv(r"C:\Users\jphos\Documents\Metis\station_geo.csv", index = False, sep=',', encoding='utf-8')

In [22]:
#Import MTA station zip code data and create dataframe
mta_zip = pd.read_csv(r"C:\Users\jphos\Documents\Metis\station_geo_out.csv")
mta_zip.head()

,LAT,LON,STATION,zipcode
0,40.733354,-73.870983,WOODHAVEN BLVD & QUEENS BLVD AT SW CORNER,11373
1,40.733041,-73.870334,WOODHAVEN BLVD & QUEENS BLVD AT SE CORNER,11373
2,40.693451,-73.852556,WOODHAVEN BLVD & JAMAICA AVE AT SW CORNER,11421
3,40.693563,-73.852608,WOODHAVEN BLVD & JAMAICA AVE AT NW CORNER,11421
4,40.706385,-74.009658,WILLIAM ST & WALL ST AT SW CORNER,10005


In [23]:
#Create file for use with MTA data notebook
with open('mta_station_zip.pickle', 'wb') as to_write:
    pickle.dump(mta_zip, to_write)

In [25]:
#Dedupe on zipcode
mta_zip.sort_values(["zipcode"], inplace=True, \
                          ascending=False)
mta_zip.drop_duplicates(["zipcode"], inplace=True)
mta_zip.head()

,LAT,LON,STATION,zipcode
378,0.551045,-74.1514904,NELSON AV & BROWER ST AT NE CORNER,83010
53,0.688157,-73.965579,WASHINGTON AVE & LAFAYETTE AVE AT SE CORNER,82010
1096,40.583369,-73.827474,BEACH 106TH ST & ROCKAWAY FRWY AT SW CORNER,11694
1077,40.588080,-73.813907,BEACH 89TH ST & ROCKAWAY FRWY AT NW CORNER,11693
1079,40.590744,-73.797789,BEACH 67TH ST & ROCKAWAY FRWY AT NW CORNER,11692


In [26]:
#Create file for use with IRS Donor data notebook
with open('mta_zip_unique.pickle', 'wb') as to_write:
    pickle.dump(mta_zip, to_write)